In [6]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.preprocessing import LabelEncoder
import math
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
import warnings
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [7]:
class ImageDataset(Dataset):
    def __init__(self, images, labels=None,transforms=None):
        self.images = images
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if self.transforms:
           image = self.transforms(Image.fromarray(self.images[idx]))
        else:
            image = torch.FloatTensor(self.images[idx]).permute(2, 0, 1)
        if self.labels is not None:
            return image, torch.LongTensor([self.labels[idx]])[0]
        return image

In [8]:
''' class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.3)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)
        x = self.dropout1(F.relu(self.fc1(x)))
        x = self.dropout2(F.relu(self.fc2(x)))
        return self.fc3(x)
 '''
import torch
import torch.nn as nn
import torch.nn.functional as F
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn
                .Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    def forward(self, x):
          out = F.relu(self.bn1(self.conv1(x)))
          out = self.bn2(self.conv2(out))
          out += self.shortcut(x)
          out = F.relu(out)
          return out
class SmallResNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.GELU()
        )
        self.layer2 = nn.Sequential(ResBlock(32, 32), ResBlock(32, 32))
        self.layer3 = nn.Sequential(ResBlock(32, 64, stride=2), ResBlock(64, 64))
        self.layer4 = nn.Sequential(ResBlock(64, 128, stride=2), ResBlock(128, 128))
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        elif isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.gap(x).squeeze(-1).squeeze(-1)
        x = self.fc(x)
        return x


In [14]:
def load_image(path):
    img = cv2.imread(path)
    if img is None: return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Resize the image to a fixed size (e.g., 224x224)
    img = cv2.resize(img, (224, 224))
    return img  # trả về uint8, đã resize, chưa /255
def load_data(csv_path, image_dir, has_labels=True):
    df = pd.read_csv(csv_path)
    images, labels = [], []

    for idx, row in df.iterrows():
        img = load_image(os.path.join(image_dir, row['image']))
        if img is not None:
            images.append(img)
            if has_labels:
                labels.append(row['label'])
        if (idx + 1) % 100 == 0:
            print(f"Đã tải {idx + 1}/{len(df)} ảnh")

    print(f"Tải thành công {len(images)} ảnh")
    if has_labels:
        return np.array(images), np.array(labels)
    return np.array(images), df['image'].values

In [12]:
def train_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Thiết bị: {device}")

    # Tải dữ liệu
    X_train, y_train = load_data('/content/drive/MyDrive/Content/CUB/train.csv', '/content/drive/MyDrive/Content/ALL_IMAGES')

    # Mã hóa nhãn
    encoder = LabelEncoder()
    y_encoded = encoder.fit_transform(y_train)
    num_classes = len(encoder.classes_)
    print(f"Số lớp: {num_classes}")

    # Chia train/val
    X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

    #Tinh mean/std
    mean = np.mean(X_tr, axis=(0,1,2)) / 255.0  # vì X_tr lúc này là uint8
    std  = np.std(X_tr,  axis=(0,1,2)) / 255.0
    print(f"Mean: {mean}, Std: {std}")


    #Transform
    train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.RandomErasing(p=0.25)
    ])

    val_tf = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean, std),
    ])
    test_tf = val_tf

    # Tạo dataloader
    train_loader = DataLoader(ImageDataset(X_tr, y_tr, transforms=train_tf), batch_size=16, shuffle=True,num_workers=2,pin_memory=True)
    val_loader = DataLoader(ImageDataset(X_val, y_val, transforms=val_tf), batch_size=16, shuffle=False,num_workers=2,pin_memory=True)

    # Khởi tạo mô hình
    model = SmallResNet(num_classes).to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.02)

    def lr_lambda(epoch):
      warmup, total = 5, 50
      if epoch < warmup:
          return float(epoch + 1)/warmup
      import math
      t = (epoch - warmup)/(total - warmup)
      return 0.5*(1 + math.cos(math.pi*t))

    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
    scaler = torch.cuda.amp.GradScaler()
    print("Bắt đầu huấn luyện...")
    for epoch in range(50):
        model.train()
        train_loss, correct, total = 0.0, 0, 0

        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)

            # ✅ chỉ 1 lần forward với AMP
            with torch.cuda.amp.autocast():
                output = model(data)
                loss = criterion(output, target)

            # ✅ backward + clip + step với AMP
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            _, pred = torch.max(output, 1)
            total += target.size(0)
            correct += (pred == target).sum().item()

            if batch_idx % 10 == 0:
                print(f'Epoch {epoch+1}/50, Batch {batch_idx}, '
                      f'Loss: {loss.item():.4f}, Acc: {100.*correct/total:.2f}%')

        # ✅ step scheduler SAU mỗi epoch
        scheduler.step()

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            # (tuỳ chọn) dùng AMP ở eval để tăng tốc
            for data, target in val_loader:
                data = data.to(device, non_blocking=True)
                target = target.to(device, non_blocking=True)
                with torch.cuda.amp.autocast():
                    output = model(data)
                    vloss = criterion(output, target)
                val_loss += vloss.item()
                _, pred = torch.max(output, 1)
                val_total += target.size(0)
                val_correct += (pred == target).sum().item()

        print(f'Epoch {epoch+1} - Train Loss: {train_loss/len(train_loader):.4f}, '
              f'Train Acc: {100.*correct/total:.2f}%, '
              f'Val Loss: {val_loss/len(val_loader):.4f}, '
              f'Val Acc: {100.*val_correct/val_total:.2f}%')
    # Lưu mô hình
    torch.save(model.state_dict(), 'cnn_model_pytorch.pth')

    # Dự đoán
    X_test, names = load_data('/content/drive/MyDrive/Content/CUB/test.csv', '/content/drive/MyDrive/Content/ALL_IMAGES', has_labels=False)
    test_loader = DataLoader(ImageDataset(X_test,transforms=test_tf), batch_size=16, shuffle=False,num_workers=2)

    model.eval()
    predictions = []
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            output = model(data)
            _, pred = torch.max(output, 1)
            predictions.extend(pred.cpu().numpy())

    # Lưu kết quả
    pred_labels = encoder.inverse_transform(predictions)
    result = pd.DataFrame({'image': names, 'label': pred_labels})
    result.to_csv('prediction.csv', index=False)

    print("Hoàn thành!")
    print(f"Đã lưu {len(predictions)} dự đoán vào prediction.csv")

In [ ]:
if __name__ == "__main__":
    train_model()

Thiết bị: cuda
Đã tải 100/900 ảnh
Đã tải 200/900 ảnh
Đã tải 300/900 ảnh
Đã tải 400/900 ảnh
Đã tải 500/900 ảnh
Đã tải 600/900 ảnh
Đã tải 700/900 ảnh
Đã tải 800/900 ảnh
Đã tải 900/900 ảnh
Tải thành công 900 ảnh
Số lớp: 30
Mean: [0.48047017 0.50314857 0.45571304], Std: [0.24075625 0.23772563 0.27127864]
Bắt đầu huấn luyện...
Epoch 1/50, Batch 0, Loss: 4.4891, Acc: 0.00%
Epoch 1/50, Batch 10, Loss: 3.7841, Acc: 3.41%
Epoch 1/50, Batch 20, Loss: 3.8434, Acc: 3.57%
Epoch 1/50, Batch 30, Loss: 3.9349, Acc: 4.03%
Epoch 1/50, Batch 40, Loss: 4.0278, Acc: 4.12%
Epoch 1 - Train Loss: 3.8942, Train Acc: 4.72%, Val Loss: 3.5293, Val Acc: 8.89%
Epoch 2/50, Batch 0, Loss: 3.6393, Acc: 0.00%
Epoch 2/50, Batch 10, Loss: 3.5839, Acc: 3.41%
Epoch 2/50, Batch 20, Loss: 3.5399, Acc: 3.87%
Epoch 2/50, Batch 30, Loss: 3.2768, Acc: 4.64%
Epoch 2/50, Batch 40, Loss: 3.6906, Acc: 5.34%
Epoch 2 - Train Loss: 3.6024, Train Acc: 5.28%, Val Loss: 3.4332, Val Acc: 10.56%
Epoch 3/50, Batch 0, Loss: 3.5495, Acc: 6.25%